In [45]:
import pandas as pd
import numpy as np
from Project.Database import Db
from Project._03FeatureSelection.NZERTF_dataframe_redundancy_functions import find_average_power_consumption_per_minute
from Project._05InferKnowledgeOfRules.infer_rules_functions import SE_time_df, json_to_dataframe

In [46]:
app_stats = SE_time_df(json_to_dataframe(year=2, level=1, with_redundancy=False))
app_stats

{'Load_StatusSensHeatPrntAUP':     TotalAbsSupport  AbsSupport  EventCount  ExternalUtility  RelSupport  \
 0               360         360        1138         0.991781    0.986301   
 1               361         361        1138         0.991781    0.989041   
 2               361         361        1138         0.991781    0.989041   
 3               361         361        1138         0.991781    0.989041   
 4               361         361        1138         0.991781    0.989041   
 5               361         361        1138         0.991781    0.989041   
 6               361         361        1138         0.991781    0.989041   
 7               105         105        1138         0.991781    0.287671   
 8               104         104        1138         0.991781    0.284932   
 9                 1           1        1138         0.991781    0.002740   
 10                0           0        1138         0.991781    0.000000   
 11                0           0        1138  

In [47]:
average_power_consumption_per_minute_df = find_average_power_consumption_per_minute()
average_power_consumption_per_minute_df

,PowerConsumption(Wm)
Load_StatusApplianceCooktop,1496.0
Load_StatusApplianceDishwasher,282.0
Load_StatusApplianceOven,771.0
Load_StatusApplianceRangeHood,78.0
Load_StatusBA1Lights,48.0
Load_StatusBA2Lights,11.0
Load_StatusBR2Lights,40.0
Load_StatusBR3Lights,33.0
Load_StatusBR4Lights,27.0
Load_StatusDRLights,38.0


In [103]:
matrix_dict={}
for key, value in app_stats.items():
    lifespan = value['Lifespan'][0]
    hour_slots = int(np.ceil(lifespan/60))

    average_power_consumption_per_minute = average_power_consumption_per_minute_df.loc[lambda self: self.index == key].values[0][0]

    hour_list = [0 for n in range(24)]
    for i in range(hour_slots):
        hour_list[i] = round(average_power_consumption_per_minute*lifespan/60_000/hour_slots, 4)

    temp_dict={}
    for n in range(24+1-hour_slots):
        rotate_list = [hour_list[(i - n) % len(hour_list)] for i, x in enumerate(hour_list)]
        temp_dict.update({n: rotate_list})

    matrix_dict.update({key: pd.DataFrame(temp_dict)})#.loc[:,lambda self: self.sum(0) == self.T.dot(app_stats[key]['TimeAssociation'])]})

matrix_dict

{'Load_StatusSensHeatPrntAUP':         0       1       2       3       4       5       6       7       8   \
 0   0.0642  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000   
 1   0.0642  0.0642  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000   
 2   0.0642  0.0642  0.0642  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000   
 3   0.0642  0.0642  0.0642  0.0642  0.0000  0.0000  0.0000  0.0000  0.0000   
 4   0.0642  0.0642  0.0642  0.0642  0.0642  0.0000  0.0000  0.0000  0.0000   
 5   0.0642  0.0642  0.0642  0.0642  0.0642  0.0642  0.0000  0.0000  0.0000   
 6   0.0642  0.0642  0.0642  0.0642  0.0642  0.0642  0.0642  0.0000  0.0000   
 7   0.0000  0.0642  0.0642  0.0642  0.0642  0.0642  0.0642  0.0642  0.0000   
 8   0.0000  0.0000  0.0642  0.0642  0.0642  0.0642  0.0642  0.0642  0.0642   
 9   0.0000  0.0000  0.0000  0.0642  0.0642  0.0642  0.0642  0.0642  0.0642   
 10  0.0000  0.0000  0.0000  0.0000  0.0642  0.0642  0.0642  0.0642  0.0642   
 11  0.0000  0.0000  0

In [50]:
production = Db.load_data(consumption=False, production=True, year=2)

In [95]:
production_vector = production.head(24).groupby(lambda self: self.hour).sum()['CO2(Grams)/kWh']

In [73]:
production_vector

0     734.303195
1     752.404834
2     737.093200
3     733.296185
4     731.040668
5     850.735967
6     720.714269
7     626.421805
8     653.875044
9     617.209215
10    733.296185
11    723.538577
12    710.935000
13    710.935000
14    679.650804
15    736.087673
16    786.818003
17    748.630148
18    705.170400
19    588.369806
20    645.430512
21    537.817672
22    539.826845
23    454.902000
Name: CO2(Grams)/kWh, dtype: float64

In [104]:
matrix_dict['Load_StatusPlugLoadVacuum'].T.dot(production_vector)#[lambda self: self == self.min()]

0     801.781640
1     803.286290
2     792.980995
3     789.716865
4     853.052139
5     847.483112
6     726.510485
7     690.464091
8     685.495741
9     728.327562
10    785.670987
11    773.611600
12    766.814491
13    749.942924
14    763.507761
15    821.303031
16    828.067188
17    784.034636
18    697.606233
19    665.388511
20    638.125746
21    581.173688
22    536.457266
dtype: float64